# Notebook implementation


!pip install labelme tensorflow opencv-python matplotlib albumentations

In [2]:
import os
import time
import uuid
import cv2

In [9]:
os.getcwd()

'd:\\Deep Learning\\Face detection model\\notebook'

In [10]:
IMAGES_PATH = os.path.join('../data','images')
number_images = 30

In [13]:
cap = cv2.VideoCapture(0)
for imgnum in range(number_images):
    print('Collectingh Images {}'.format(imgnum))
    ret, frame = cap.read()
    imgname = os.path.join(IMAGES_PATH,f'{str(uuid.uuid1())}.jpg')
    cv2.imwrite(imgname, frame)
    cv2.imshow('frame', frame)
    time.sleep(0.5)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Collectingh Images 0
Collectingh Images 1
Collectingh Images 2
Collectingh Images 3
Collectingh Images 4
Collectingh Images 5
Collectingh Images 6
Collectingh Images 7
Collectingh Images 8
Collectingh Images 9
Collectingh Images 10
Collectingh Images 11
Collectingh Images 12
Collectingh Images 13
Collectingh Images 14
Collectingh Images 15
Collectingh Images 16
Collectingh Images 17
Collectingh Images 18
Collectingh Images 19
Collectingh Images 20
Collectingh Images 21
Collectingh Images 22
Collectingh Images 23
Collectingh Images 24
Collectingh Images 25
Collectingh Images 26
Collectingh Images 27
Collectingh Images 28
Collectingh Images 29


# Labeling images

In [14]:
!labelme

2024-12-08 11:47:08,925 [INFO   ] __init__:get_config:67- Loading config file from: C:\Users\radhe shayam\.labelmerc


# REVIEW DATASET

In [18]:
import tensorflow as tf
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt